In [ ]:
!pip install jira
from jira import JIRA
import json
jira = JIRA(
    server="https://khemanth.atlassian.net/",
    basic_auth=()

# def fetch_ticket(ticket_id: str) -> dict:
#     issue = jira.issue(ticket_id)
#     issue_id=ticket_id
#     issue = jira.issue(issue_id)

#     # === Convert All Fields to JSON ===
#     issue_data = {
#         "id": issue.id,
#         "key": issue.key,
#         "fields": {}
#     }

#     for field_name in issue.raw['fields']:
#         issue_data["fields"][field_name] = issue.raw['fields'][field_name]

#     # Print JSON nicely
#     print(json.dumps(issue_data, indent=2))
# fetch_ticket("SCRUM-1")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
def get_issue_details(ticket_id: str):
    issue = jira.issue(ticket_id)
    print('hi')
    # Extract comments
    comments = []
    for c in issue.fields.comment.comments:
        comments.append({
            "author": c.author.displayName,
            "body": c.body,
            "created": c.created
        })

    # Extract sprint info (if present)
    sprint_info = None
    if hasattr(issue.fields, 'customfield_10020'):  # common sprint custom field
        if issue.fields.customfield_10020:
            sprint_info = [sprint.name for sprint in issue.fields.customfield_10020]

    # Extract team name (if custom field exists)
    team_name = None
    if hasattr(issue.fields, 'customfield_10021'):  # replace with actual team name field ID
        team_name = issue.fields.customfield_10021

    # Build dictionary
    details = {
        "id": issue.id,
        "key": issue.key,
        "summary": issue.fields.summary,
        "description": issue.fields.description,
        "status": issue.fields.status.name,
        "priority": issue.fields.priority.name if issue.fields.priority else None,
        "assignee": issue.fields.assignee.displayName if issue.fields.assignee else None,
        "reporter": issue.fields.reporter.displayName if issue.fields.reporter else None,
        "labels": issue.fields.labels,
        "created": issue.fields.created,
        "updated": issue.fields.updated,
        "comments": comments,
        "sprint": sprint_info,
        "team_name": team_name
    }

    return details
print(get_issue_details("SCRUM-1"))

hi
{'id': '10003', 'key': 'SCRUM-1', 'summary': 'need to complete development of the llm code', 'description': None, 'status': 'Idea', 'priority': 'Medium', 'assignee': 'Ravi sankar', 'reporter': 'nandamnagasivani', 'labels': [], 'created': '2025-08-14T11:46:40.016+0530', 'updated': '2025-08-15T08:45:22.463+0530', 'comments': [{'author': 'Hemanth krishna', 'body': 'Hi SM,\n\nIm working on it will complete by EOD', 'created': '2025-08-14T11:47:55.190+0530'}, {'author': 'Ravi sankar', 'body': 'Hi Hemanth \nIm also looking to support you on this work please let me know if any assistance needed.\n\nRegards,\nRavi sankar', 'created': '2025-08-14T11:58:03.568+0530'}, {'author': 'Hemanth krishna', 'body': 'Hi \ncode development is completed please review the code and merge it main branch\nregards\nHemanth', 'created': '2025-08-15T08:45:22.463+0530'}], 'sprint': None, 'team_name': None}


In [ ]:
# install required libraries
!pip install transformers accelerate torch sentencepiece bitsandbytes --quiet
print("Libraries installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.7 MB/s eta 0:00:00
Libraries installed!


In [ ]:
# Log in to Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

Model selction and formating

In [ ]:
# # --- Load Gemma 2 2B ---
model_id_gemma = "google/gemma-2-2b-it"
hf_token = "hf_your_actual_token_here"
tokenizer_gemma = AutoTokenizer.from_pretrained(model_id_gemma)

model_gemma = AutoModelForCausalLM.from_pretrained(
    model_id_gemma,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
    device_map="auto"
)
print(f"Gemma 2 2B loaded")
model_gemma.save_pretrained("gemma_model_4bit")

# Save tokenizer (important for reloading)
tokenizer_gemma.save_pretrained("gemma_model_4bit")

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Gemma 2 2B loaded


('gemma_model_4bit/tokenizer_config.json',
 'gemma_model_4bit/special_tokens_map.json',
 'gemma_model_4bit/chat_template.jinja',
 'gemma_model_4bit/tokenizer.model',
 'gemma_model_4bit/added_tokens.json',
 'gemma_model_4bit/tokenizer.json')

In [ ]:
# gemini verification
print("Running quick verification...")
_prompt = "Hi! How are you today?"
_inputs = tokenizer_gemma(_prompt, return_tensors="pt").to(model_gemma.device)
_outputs = model_gemma.generate(**_inputs, max_new_tokens=50)

print("Gemma Verification Ok:", tokenizer_gemma.decode(_outputs[0]))

# cleaning up temporary variables
del _prompt, _inputs, _outputs

Running quick verification...
Gemma Verification Ok: <bos>Hi! How are you today? 😊 

I'm doing well, thank you! How about you? 
<end_of_turn>


In [ ]:

model=model_gemma
tokenizer=tokenizer_gemma
ticket_data=get_issue_details("SCRUM-1")
ticket_context = "\n".join(f"{k}: {v}" for k, v in ticket_data.items())

# 3. Create pipeline
chatbot = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

# 4. Ask a question about the ticket
question = "What is the issue and what was the latest status"
prompt = f"""You are a helpful assistant. Here is the JIRA ticket info:

{ticket_context}

Now answer the question: {question}
"""

# 5. Generate answer
response = chatbot(prompt, max_new_tokens=200, temperature=0.2)
print(response[0]["generated_text"])


Device set to use cuda:0


hi
You are a helpful assistant. Here is the JIRA ticket info:

id: 10003
key: SCRUM-1
summary: need to complete development of the llm code
description: None
status: Idea
priority: Medium
assignee: Ravi sankar
reporter: nandamnagasivani
labels: []
created: 2025-08-14T11:46:40.016+0530
updated: 2025-08-15T08:45:22.463+0530
comments: [{'author': 'Hemanth krishna', 'body': 'Hi SM,\n\nIm working on it will complete by EOD', 'created': '2025-08-14T11:47:55.190+0530'}, {'author': 'Ravi sankar', 'body': 'Hi Hemanth \nIm also looking to support you on this work please let me know if any assistance needed.\n\nRegards,\nRavi sankar', 'created': '2025-08-14T11:58:03.568+0530'}, {'author': 'Hemanth krishna', 'body': 'Hi \ncode development is completed please review the code and merge it main branch\nregards\nHemanth', 'created': '2025-08-15T08:45:22.463+0530'}]
sprint: None
team_name: None

Now answer the question: What is the issue and what was the latest status
**Answer:**

The issue is the deve

In [ ]:

!zip -r my_folder.zip /content/gemma_model_4bit
from google.colab import files
files.download("my_folder.zip")


  adding: content/gemma_model_4bit/ (stored 0%)
  adding: content/gemma_model_4bit/chat_template.jinja (deflated 52%)
  adding: content/gemma_model_4bit/config.json (deflated 68%)
  adding: content/gemma_model_4bit/tokenizer.json (deflated 84%)
  adding: content/gemma_model_4bit/tokenizer.model (deflated 51%)
  adding: content/gemma_model_4bit/model.safetensors (deflated 17%)
  adding: content/gemma_model_4bit/generation_config.json (deflated 30%)
  adding: content/gemma_model_4bit/special_tokens_map.json (deflated 76%)
  adding: content/gemma_model_4bit/tokenizer_config.json (deflated 96%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.types import State
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ----------------------------
# Load Hugging Face model
# ----------------------------
model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# ----------------------------
# Ticket context
# ----------------------------
ticket_data = {
    "ticket_id": "JIRA-1234",
    "summary": "User unable to access dashboard",
    "status": "In Progress",
    "assignee": "John Doe",
    "reporter": "Alice Smith",
    "latest_update": "Backend team investigating API latency",
    "labels": ["dashboard", "access", "latency"],
    "priority": "High"
}
ticket_context = "\n".join(f"{k}: {v}" for k, v in ticket_data.items())

# ----------------------------
# LangGraph state
# ----------------------------
class ChatState(State):
    history: list  # list of (question, answer)
    context: str   # ticket info

# ----------------------------
# Model node
# ----------------------------
def answer_question(state: ChatState) -> ChatState:
    # Build prompt from ticket context + history
    conversation = f"You are a helpful assistant. Here is the JIRA ticket info:\n\n{state.context}\n"
    for q, a in state.history:
        conversation += f"\nUser: {q}\nAssistant: {a}"
    # The last question to answer
    last_question = state.history[-1][0]
    conversation += f"\nUser: {last_question}\nAssistant:"

    # Run model
    response = llm_pipeline(conversation, max_new_tokens=200, temperature=0.2)
    answer = response[0]["generated_text"].split("Assistant:")[-1].strip()

    # Replace last tuple with answer
    state.history[-1] = (last_question, answer)
    return state

# ----------------------------
# Build LangGraph flow
# ----------------------------
graph = StateGraph(ChatState)
graph.add_node("answer_question", answer_question)
graph.set_entry_point("answer_question")
graph.add_edge("answer_question", END)
app = graph.compile()

# ----------------------------
# Chat loop with memory
# ----------------------------
state = ChatState(history=[], context=ticket_context)

def ask(question):
    state.history.append((question, ""))  # add question with no answer yet
    result = app.invoke(state)
    return result.history[-1][1]  # return the answer

# Example usage
print(ask("Who is responsible for fixing this issue?"))
print(ask("What is the latest update?"))
print(ask("Is the issue high priority?"))


ModuleNotFoundError: No module named 'langgraph'

In [ ]:
import streamlit as st
import time

# --- Mock function for model response ---
# Replace this with your Jira + LangGraph + LLM pipeline call
def get_model_response(question):
    # Example response from your AI model
    return "The latest update on ticket ABC-123 is that it is currently assigned to John Doe and awaiting QA approval."

# --- Streamlit UI ---
st.set_page_config(page_title="Jira Smart Assistant", page_icon="🤖")

st.title("🤖 Jira Smart Assistant")
st.write("Ask me anything about your Jira tickets.")

# User input
user_question = st.text_input("Enter your question:", placeholder="e.g., What's the latest update on ABC-123?")

# Submit button
if st.button("Ask"):
    if user_question.strip():
        # Get model's full answer
        full_answer = get_model_response(user_question)

        # Simulate ChatGPT-style word-by-word display
        placeholder = st.empty()
        displayed_text = ""
        for word in full_answer.split():
            displayed_text += word + " "
            placeholder.markdown(displayed_text)
            time.sleep(0.05)  # delay for streaming effect
    else:
        st.warning("Please enter a question.")


2025-08-15 16:08:08.826 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 16:08:08.827 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 16:08:08.884 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-15 16:08:08.885 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 16:08:08.887 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 16:08:08.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-15 16:08:08.889 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.7 MB/s eta 0:00:00
